# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# XT3D Example 0 -- Using XT3D for More Accurate Flows on an Unstructured Grid and with Anisotropy

This example is based on the MODFLOW 6 "Nested Grid" ([ex-gwf-u1disv](https://modflow6-examples.readthedocs.io/en/latest/_examples/ex-gwf-u1disv.html)) example problem, which corresponds to the first example described in the MODFLOW-USG documentation.

The DISV grid in this example is a rectangular grid with an area of quadtree refinement in the middle. Some of the connections between cells in this grid violate the so-called "CVFD requirements," rendering the standard, conductance-based flow formulation less accurate. In the first part of this example we'll demonstrate the improved accuracy the XT3D flow formulation can provide in flow simulations on unstructured grids such as this one.

In the second part of this example, we'll add anisotropy and see how XT3D can also help when the principal directions (the directions of maximum and minimum hydraulic conductivity) are *not* aligned with the directions of the grid connections.

### Preliminaries

In [ ]:
import math
import pathlib as pl

import flopy
import flopy.utils.cvfdutil
import matplotlib.pyplot as plt
import numpy as np

# Example name and workspace paths
example_name = "xt3d-ex0-unstructured-grid"
sim_ws = pl.Path(example_name)
figs_path = sim_ws

### Set some parameters

In [ ]:
# Model units
length_units = "meters"
time_units = "days"

strt = 0.0  # Starting head (m)
icelltype = 0  # Cell conversion type
k11 = 1.0  # Horizontal hydraulic conductivity (m / d)

xt3doptions = None  # Base case is without xt3d

### TDIS data
Simulation has 1 steady stress period (1 day)
and 3 transient stress periods (10 days each).
Each transient stress period has 120 2-hour time steps.

In [ ]:
nper = 1  # Number of periods
perlen = [1.0]  # period length
nstp = [1]  # nr. of timesteps
tsmult = [1.0, 1.0, 1.0]  # timestep multiplier
tdis_ds = list(zip(perlen, nstp, tsmult))  # data set for TDIS

### Outer grid
Start with creating the outer grid as a FloPy structured grid object. Below we will create the inner grid and then merge the two in a single unstructured discretization-by-vertices (DISV) grid.

In [ ]:
nlay = 1
nrow = ncol = 7
top = 0.0
botm = -100.0
delr = 100.0 * np.ones(ncol)
delc = 100.0 * np.ones(nrow)
tp = np.zeros((nrow, ncol))
bt = -100.0 * np.ones((nlay, nrow, ncol))

Set idomain to zero (inactive) where the refined inset grid will be positioned.

In [ ]:
idomain = np.ones((nlay, nrow, ncol))
idomain[:, 2:5, 2:5] = 0

Create the outer grid.

In [ ]:
sg1 = flopy.discretization.StructuredGrid(
    delr=delr, delc=delc, top=tp, botm=bt, idomain=idomain
)

### Inner grid

Set geometry data.

In [ ]:
nlay = 1
nrow = ncol = 9
delr = 100.0 / 3.0 * np.ones(ncol)
delc = 100.0 / 3.0 * np.ones(nrow)
tp = np.zeros((nrow, ncol))
bt = -100 * np.ones((nlay, nrow, ncol))

All cells are active so set idomain to 1 everywhere.

In [ ]:
idomain = np.ones((nlay, nrow, ncol))

Create the grid inner, refined structured grid object.

In [ ]:
sg2 = flopy.discretization.StructuredGrid(
    delr=delr,
    delc=delc,
    top=tp,
    botm=bt,
    xoff=200.0,
    yoff=200,
    idomain=idomain,
)

The following utility function will convert the two FloPy structured grid object into a data structure that can be passed directly into the DISV package to create the full unstructured grid.

In [ ]:
gridprops = flopy.utils.cvfdutil.gridlist_to_disv_gridprops([sg1, sg2])
print("Ignore this warning for now")

### Solver parameters

In [ ]:
nouter = 50
ninner = 100
hclose = 1e-9
rclose = 1e-6

### Model setup

Now we are ready to set up the FloPy simulation model.

In [ ]:
sim_name = example_name
model_name = "gwf"

In [ ]:
def hgrad(k, k22, angle1, qx, qy):
    angle = angle1 * math.pi / 180

    kxx = k * math.cos(angle) ** 2 + k22 * math.sin(angle) ** 2
    kyy = k * math.sin(angle) ** 2 + k22 * math.cos(angle) ** 2
    kxy = (k - k22) * math.sin(angle) * math.cos(angle)
    kyx = kxy
    det = kxx * kyy - kxy * kyx
    kinvxx = kyy / det
    kinvxy = -kxy / det
    kinvyx = -kyx / det
    kinvyy = kxx / det
    hgradx = -kinvxx * qx - kinvxy * qy
    hgrady = -kinvyx * qx - kinvyy * qy

    return hgradx, hgrady

In [ ]:
def add_chds(gwf, hgradx=None, hgrady=None):
    if hgradx is None:
        chd_spd = []
        chd_spd += [[0, i, 1.0] for i in [0, 7, 14, 18, 22, 26, 33]]
        chd_spd = {0: chd_spd}
        flopy.mf6.ModflowGwfchd(
            gwf,
            stress_period_data=chd_spd,
            pname="CHD-LEFT",
            filename=f"{model_name}.left.chd",
        )

        chd_spd = []
        chd_spd += [[0, i, 0.0] for i in [6, 13, 17, 21, 25, 32, 39]]
        chd_spd = {0: chd_spd}
        flopy.mf6.ModflowGwfchd(
            gwf,
            stress_period_data=chd_spd,
            pname="CHD-RIGHT",
            filename=f"{model_name}.right.chd",
        )

    else:
        chd_spd = []
        hconst = hgradx * 50 + hgrady * 50 - 1
        for irow, i in enumerate([0, 7, 14, 18, 22, 26, 33]):
            x = 50
            y = (6 - irow) * 100 + 50  # hardwired dimensions
            head = hgradx * x + hgrady * y - hconst
            chd_spd += [[0, i, head]]
        chd_spd = {0: chd_spd}
        flopy.mf6.ModflowGwfchd(
            gwf,
            stress_period_data=chd_spd,
            pname="CHD-LEFT",
            filename=f"{model_name}.left.chd",
        )
        chd_spd = []
        for irow, i in enumerate([6, 13, 17, 21, 25, 32, 39]):
            x = 650
            y = (6 - irow) * 100 + 50  # hardwired dimensions
            head = hgradx * x + hgrady * y - hconst
            chd_spd += [[0, i, head]]
        chd_spd = {0: chd_spd}
        flopy.mf6.ModflowGwfchd(
            gwf,
            stress_period_data=chd_spd,
            pname="CHD-RIGHT",
            filename=f"{model_name}.right.chd",
        )
        chd_spd = []
        for jcol, i in enumerate([1, 2, 3, 4, 5]):
            x = jcol * 100 + 150  # hardwired dimensions
            y = 650
            head = hgradx * x + hgrady * y - hconst
            chd_spd += [[0, i, head]]
        chd_spd = {0: chd_spd}
        flopy.mf6.ModflowGwfchd(
            gwf,
            stress_period_data=chd_spd,
            pname="CHD-TOP",
            filename=f"{model_name}.top.chd",
        )
        chd_spd = []
        for jcol, i in enumerate([34, 35, 36, 37, 38]):
            x = jcol * 100 + 150  # hardwired dimensions
            y = 50
            head = hgradx * x + hgrady * y - hconst
            chd_spd += [[0, i, head]]
        chd_spd = {0: chd_spd}
        flopy.mf6.ModflowGwfchd(
            gwf,
            stress_period_data=chd_spd,
            pname="CHD-BOTTOM",
            filename=f"{model_name}.bottom.chd",
        )

In [ ]:
def build_sim(hgradx=None, hgrady=None):
    sim = flopy.mf6.MFSimulation(
        sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6"
    )
    flopy.mf6.ModflowTdis(
        sim, nper=nper, perioddata=tdis_ds, time_units=time_units
    )
    flopy.mf6.ModflowIms(
        sim,
        linear_acceleration="bicgstab",
        outer_maximum=nouter,
        outer_dvclose=hclose,
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=f"{rclose} strict",
    )
    gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name, save_flows=True)
    flopy.mf6.ModflowGwfdisv(
        gwf,
        length_units=length_units,
        nlay=nlay,
        top=top,
        botm=botm,
        **gridprops,
    )
    flopy.mf6.ModflowGwfnpf(
        gwf,
        icelltype=icelltype,
        k=k11,
        save_specific_discharge=True,
        xt3doptions=xt3doptions,
    )
    flopy.mf6.ModflowGwfic(gwf, strt=strt)

    add_chds(gwf, hgradx=hgradx, hgrady=hgrady)

    head_filerecord = f"{model_name}.hds"
    budget_filerecord = f"{model_name}.cbc"
    flopy.mf6.ModflowGwfoc(
        gwf,
        head_filerecord=head_filerecord,
        budget_filerecord=budget_filerecord,
        saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    )

    return sim

### Function to plot head and specific discharge results

In [ ]:
def plot_results(hgradx=None, hgrady=None):
    # Get the head data from the model
    gwf = sim.get_model(model_name)
    head = gwf.output.head().get_data()[:, 0, :]

    # Get the specific discharge vector from the budget data object
    qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(
        gwf.output.budget().get_data(text="DATA-SPDIS", totim=1.0)[0],
        gwf,
    )

    # Create the figure
    fig = plt.figure(figsize=(14, 8))
    fig.tight_layout()

    ax = fig.add_subplot(1, 2, 1, aspect="equal")
    pmv = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0)
    pmv.plot_grid()
    cb = pmv.plot_array(head, cmap="jet")
    pmv.plot_vector(
        qx,
        qy,
        normalize=False,
        color="0.25",
    )
    plt.colorbar(cb, shrink=0.6)
    ax.set_xlabel("x position (m)")
    ax.set_ylabel("y position (m)")
    ax.set_title("Simulated Head")

    ax = fig.add_subplot(1, 2, 2, aspect="equal")
    pmv = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0)
    pmv.plot_grid()
    if hgradx is None:
        x = np.array(gwf.modelgrid.xcellcenters) - 50.0
        slp = (1.0 - 0.0) / (50.0 - 650.0)
        heada = slp * x + 1.0
    else:
        hconst = hgradx * 50 + hgrady * 50 - 1
        x = np.array(gwf.modelgrid.xcellcenters)
        y = np.array(gwf.modelgrid.ycellcenters)
        heada = hgradx * x + hgrady * y - hconst
    cb = pmv.plot_array(head - heada, cmap="jet")
    plt.colorbar(cb, shrink=0.6)
    ax.set_xlabel("x position (m)")
    ax.set_ylabel("y position (m)")
    max_error = np.amax(head - heada)
    ax.set_title(f"Error (maximum = {max_error:.2g})")

### Build simulation

In [ ]:
sim = build_sim()

## Exercise A -- CVFD requirement violations in our quadtree grid

The "CVFD requirements" state that for the standard, conductance-based formulation for flow between two cells to be accurate, the connection between the cell centers (nodes) must bisect the shared face between the two cells at a right angle.

__A1.__ Using the notebook cell below, plot the model grid.

__A2.__ Geometrically, the grid connections can be represented by line segments that connect the centers of adjacent cells. (The connections aren't drawn here, but they're easy enough to imagine.) Visually identify which cell connections violate the CVFD requirements.

In [ ]:
# Figure properties
figure_size = (6, 6)

fig = plt.figure(figsize=figure_size)
fig.tight_layout()

gwf = sim.get_model(model_name)

ax = fig.add_subplot(1, 1, 1, aspect="equal")
pmv = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0)

# plot the grid
pmv.plot_grid()

# add the boundary conditions (CHD)
pmv.plot_bc(name="CHD-LEFT", alpha=0.75)
pmv.plot_bc(name="CHD-RIGHT", alpha=0.75)

ax.set_xlabel("x position (m)")
ax.set_ylabel("y position (m)")

# this plots the cell ids
for i, (x, y) in enumerate(
    zip(gwf.modelgrid.xcellcenters, gwf.modelgrid.ycellcenters)
):
    ax.text(
        x,
        y,
        f"{i + 1}",
        fontsize=6,
        horizontalalignment="center",
        verticalalignment="center",
    )

## Exercise B -- Standard, conductance-based formulation (without XT3D)

The idea of the problem setup is to create a uniform, left-to-right steady-state groundwater flow through this square domain of homogeneous, isotropic hydraulic conductivity. The flow is induced by specifying constant heads along the left and right sides (blue cells in the grid plot), with higher head on the left than on the right.

__B1.__ Use the notebook cell below to write and run the simulation __without__ XT3D, plot the results, and print the water-volume budget for the model.

__B2.__ How does the error in head and the direction of the specific discharge (groundwater flux) vectors generally correlate with the locations of connections that violate the CVFD requirements? How does the maximum error in the heads compare with the closure criterion (convergence tolerance) for the head solution (hclose)?

__B3__. The confined aquifer is 700 m by 700 m in plan view, with a thickness of 10 m and a hydraulic conductivity 1 m/d. A constant-head (CHD) value of 1 m is assigned to the cell centers (nodes) of the leftmost column of cells (at x=50 m), and a constant-head value of 0 m is assigned to the cell centers of the rightmost column (at x=650 m). Using Darcy’s Law, calculate the expected total rate of groundwater flow through the model, i.e., the rate at which water enters through the left side, which equals the rate rate at which it exits through the right side. How does the total simulated rate of groundwater flow through the model reported in the budget compare with the expected rate?

In [ ]:
gwf.npf.xt3doptions = ""
gwf.npf.write()
sim.write_simulation(silent=True)
sim.run_simulation(silent=True)
plot_results()

lstpath = sim_ws / "gwf.lst"
mf_list = flopy.utils.Mf6ListBudget(sim_ws / "gwf.lst")
budget_data = mf_list.get_data(kstpkper=(0, 0))
for item in budget_data:
    print(item["name"], item["value"])

## Exercise C -- With XT3D

__C1.__ Use the notebook cell below to write and rerun the simulation __with XT3D__ and report the new results.

__C2.__ How do the specific discharge vectors look now? How does the maximum error in the heads compare with hclose? (Note that the scale of the head error plot is very different than before.)

__C3__. How does the total simulated rate of groundwater flow through the model reported in the budget compare with the expected rate?

In [ ]:
gwf.npf.xt3doptions = "xt3d"
gwf.npf.write()
sim.write_simulation(silent=True)
sim.run_simulation(silent=True)
plot_results()

lstpath = sim_ws / "gwf.lst"
mf_list = flopy.utils.Mf6ListBudget(sim_ws / "gwf.lst")
budget_data = mf_list.get_data(kstpkper=(0, 0))
for item in budget_data:
    print(item["name"], item["value"])

## Exercise D -- Anisotropy

When the hydraulic conductivity is __isotropic__, the groundwater flux is in the same direction as the head gradient. When the hydraulic conductivity is __anisotropic__, the groundwater flux is generally not in the same direction as the head gradient (except when the head gradient aligns with one of the principal directions of anisotropy). In the case of general anisotropy, the groundwater flux and the head gradient vector are related by the following form of Darcy's Law:

$\mathbf{q} = - \mathbf{K} \nabla h$

where $\mathbf{K}$ is the hydraulic conductivity tensor. Written out in 2D, Cartesian (x, y) coordinates, this has the form

$q_x = - K_{xx} \partial h / \partial x - K_{xy} \partial h / \partial y$

$q_y = - K_{yx} \partial h / \partial x - K_{yy} \partial h / \partial y$

where $K_{yx} = K_{xy}$. Note that each component of the groundwater flux depends on both components of the head gradient.

In the NPF Package's input, $K$ is the maximum conductivity, K22 is the minimum conductivity, and ANGLE1 is the angle by the the anisotropy direction are rotated counterclockwise relative to the (x, y) corrdinate axes.  In terms of these parameters, the elements of the conductivity tensor in (x, y) coordinates are

$K_{xx} = K cos^2 \{ANGLE1\} + K22 sin^2 \{ANGLE1\}$

$K_{yy} = K sin^2 \{ANGLE1\} + K22 cos^2 \{ANGLE1\}$

$K_{xy} = K_{yx} = \left ( K - K22 \right ) sin \{ANGLE1\} cos \{ANGLE1\}$

The bottom line is that for a given anisotropy angle ANGLE1, one can calculate the elements of $\mathbf{K}$, then invert Darcy's Law to calculate the head gradient vector that corresponds to our desired unit groundwater flux in the positive $x$ direction. In the previous exercise, which featured isotropic hydraulic conductivity, the head gradient pointed in the positive $x$ direction. __In the anisotropic case with $ANGLE1$ not equal to 0, the required head gradient will point in a different direction.__

__D1.__ Use the notebook cell below to calculate the $x$ and $y$ components of the head gradient that produces the same horizontal groundwater flux as before for an anisotropy angle of 45 degrees and an anisotropy ratio of 10:1, then set up corresponding constant heads around the perimeter of the model domain, and rerun the problem.

__D2.__ Set xt3doptions=None and rerun without XT3D. Play with different anisotropy angle with and without XT3D.

In [ ]:
k = k11
k22 = 0.1 * k
angle1 = 45
qx = 1 / 600
qy = 0
xt3doptions = "xt3d"

hgradx, hgrady = hgrad(k, k22, angle1, qx, qy)
print("dh/dx = ", hgradx)
print("dh/dy = ", hgrady)

sim = build_sim(hgradx=hgradx, hgrady=hgrady)
sim.write_simulation(silent=True)
gwf = sim.get_model(model_name)
gwf.npf.xt3doptions = xt3doptions
gwf.npf.k = k
gwf.npf.k22 = k22
gwf.npf.angle1 = angle1
gwf.npf.write()
sim.run_simulation(silent=True)
plot_results(hgradx=hgradx, hgrady=hgrady)